## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-28-10-45-00 +0000,wsj,"Legendary French Actress, Sex Symbol Brigitte ...",https://www.wsj.com/arts-culture/film/legendar...
1,2025-12-28-10-35-09 +0000,nyt,"From Sex Appeal to the Far Right, Brigitte Bar...",https://www.nytimes.com/2025/12/28/arts/brigit...
2,2025-12-28-10-34-02 +0000,wapo,"Brigitte Bardot, French femme fatale and cultu...",https://www.washingtonpost.com/obituaries/2025...
3,2025-12-28-10-30-44 +0000,wapo,"A few life lessons from Robert Redford, Jane G...",https://www.washingtonpost.com/obituaries/inte...
4,2025-12-28-10-30-00 +0000,wsj,Trump’s Broadside at Europe Also Swipes at a U...,https://www.wsj.com/world/europe/trumps-broads...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2341/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
27,trump,16
112,new,9
7,dies,8
43,year,7
5,brigitte,7


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
11,2025-12-28-10-02-11 +0000,nyt,Zelensky to Meet With Trump at Mar-a-Lago Abou...,https://www.nytimes.com/2025/12/28/world/europ...,45
74,2025-12-28-00-00-00 +0000,wsj,President Trump overrode some on the religious...,https://www.wsj.com/politics/policy/how-trump-...,40
46,2025-12-28-03-35-45 +0000,nyt,Russia Pummels Kyiv Before Trump-Zelensky Meeting,https://www.nytimes.com/2025/12/27/world/europ...,39
35,2025-12-28-05-53-06 +0000,bbc,"Zelensky says Russia 'doesn't want peace', ahe...",https://www.bbc.com/news/articles/c1dzvpg1341o...,38
124,2025-12-27-14-29-52 +0000,wapo,Russian attack pummels Kyiv as Zelensky prepar...,https://www.washingtonpost.com/world/2025/12/2...,38


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
11,45,2025-12-28-10-02-11 +0000,nyt,Zelensky to Meet With Trump at Mar-a-Lago Abou...,https://www.nytimes.com/2025/12/28/world/europ...
0,34,2025-12-28-10-45-00 +0000,wsj,"Legendary French Actress, Sex Symbol Brigitte ...",https://www.wsj.com/arts-culture/film/legendar...
91,33,2025-12-27-21-27-46 +0000,nypost,"Kim Jong Un hails bond of ‘blood, life and dea...",https://nypost.com/2025/12/27/world-news/kim-j...
26,31,2025-12-28-08-59-41 +0000,nypost,Next major storm system threatens Midwest with...,https://nypost.com/2025/12/28/us-news/major-st...
53,28,2025-12-28-02-17-16 +0000,nypost,Heroic Texas father rescues abducted daughter ...,https://nypost.com/2025/12/27/us-news/heroic-t...
74,24,2025-12-28-00-00-00 +0000,wsj,President Trump overrode some on the religious...,https://www.wsj.com/politics/policy/how-trump-...
71,23,2025-12-28-00-11-17 +0000,cbc,"Amid a civil war, polls open in Myanmar for 1s...",https://www.cbc.ca/news/world/myanmar-election...
37,23,2025-12-28-05-33-21 +0000,nypost,"Australian cruise ship runs aground, strikes c...",https://nypost.com/2025/12/28/world-news/austr...
94,21,2025-12-27-20-57-25 +0000,nypost,"Iran declares ‘total war’ against US, Israel a...",https://nypost.com/2025/12/27/world-news/irani...
59,20,2025-12-28-01-47-47 +0000,nypost,Tipsy thief allegedly steals 2 mandolins from ...,https://nypost.com/2025/12/27/us-news/tipsy-th...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
